# MEGADOCK on Google Colaboratory (PDB files upload version)
**Protein-protein docking with MEGADOCK on Google Colaboratory**  
MEGADOCKによるタンパク質間ドッキングをGoogle Colaboratoryで実行する

> 💻 MEGADOCK 4   https://github.com/akiyamalab/MEGADOCK  
> 📃 Ohue M, _et al_. **_Bioinformatics_**, 30(22): 3281-3283, 2014. http://doi.org/10.1093/bioinformatics/btu532  
> License: MEGADOCK is licensed by CC BY-NC 4.0. MEGADOCK and its derivatives are NOT allowed for any commercial use without formal prior authorization by Tokyo Institute of Technology.

**Hit `Runtime` → `Run all`, then please upload your PDB files**  
`ランタイム` → `すべてのセルを実行` をクリックしてから、出てきた `ファイル選択` からPDBファイルをアップロードしてください。

Note: mmCIF files (.cif) are not supported. mmCIF files must be converted to PDB format (.pdb) by the converter.
https://mmcif.pdbj.org/converter/index.php?l=en  
注意: mmCIFファイル (.cif) には対応していません。mmCIFファイルを用いる場合は変換サービスでPDBファイル (.pdb) に変換してください。 https://mmcif.pdbj.org/converter/index.php?l=ja


In [ ]:
from google.colab import files
#@markdown **Receptor PDB** (`-R`)
print("upload receptor PDB file (.pdb)")
rup = files.upload()
rfilename = list(rup.keys())[0]

In [ ]:
#@markdown **Ligand PDB** (`-L`)
print("upload ligand PDB files (.pdb)")
lup = files.upload()
lfilename = list(lup.keys())[0]

In [3]:
#@markdown MEGADOCK parameters (if you want to change)
t = "3" #@param {type:"string"}
#D = 0 #@param {type:"string"}
N = "10800" #@param {type:"string"}
outfile_name = "dock.out" #@param {type:"string"}

In [ ]:
#@title Check GPU (nvidia-smi)
!nvidia-smi

In [ ]:
#@title Software Installation

# MEGADOCK
!git clone https://github.com/akiyamalab/MEGADOCK
!git clone https://github.com/NVIDIA/cuda-samples
!apt-get install -y libfftw3-dev libfftw3-single3

%cd ./MEGADOCK
!make -j 2 -f Makefile.colab

# biopython
!pip -q install biopython

# NGLView
!pip -q install nglview

In [ ]:
#@title MEGADOCK parameter setting
import os
os.environ['MDt'] = str(t)
os.environ['MDN'] = str(N)
os.environ['MDPDBR'] = rfilename
os.environ['MDPDBL'] = lfilename
os.environ['MDOF'] = str(outfile_name)

!bash -c "mv /content/$MDPDBR ."
!bash -c "mv /content/$MDPDBL ."

In [ ]:
#@title Run MEGADOCK
!./megadock-gpu -R $MDPDBR -L $MDPDBL -t $MDt -N $MDN -o $MDOF

In [ ]:
#@title Show the 1st solution on PDB viewer (NGLView)
!./decoygen lig.1.pdb $MDPDBL $MDOF 1
!cat $MDPDBR lig.1.pdb | sed 's/END//g' > complex1.pdb
from google.colab import output
output.enable_custom_widget_manager()
import nglview as nv
view = nv.show_structure_file("complex1.pdb")
view

In [ ]:
#@title Show the top 5 solutions on PDB viewer (NGLView)
!./decoygen lig.2.pdb $MDPDBL $MDOF 2
!./decoygen lig.3.pdb $MDPDBL $MDOF 3
!./decoygen lig.4.pdb $MDPDBL $MDOF 4
!./decoygen lig.5.pdb $MDPDBL $MDOF 5
!cat complex1.pdb lig.2.pdb lig.3.pdb lig.4.pdb lig.5.pdb | sed 's/END//g' > complex5s.pdb
from google.colab import output
output.enable_custom_widget_manager()
import nglview as nv
view = nv.show_structure_file("complex5s.pdb")
view

In [ ]:
#@title PPI score calculation (for protein-protein interaction prediction)
!./ppiscore $MDOF $MDN

The value of the PPI Score (the value of $E$ in the above cell) can predict whether two protein chains will interact or not.  
The approximate values of precision (positive predictive value, PPV) and PPI Score are shown in the Figure below.  
* The precision is about 10% when $E > 8$ is predicted as "PPI positive."
* The precision is about 50% when $E > 10$ is predicted as "PPI positive."
* The precision is about 80% when $E > 12$ is predicted as "PPI positive."

PPI Scoreの値（上のセルの $E$ の値）で2つのタンパク質鎖が相互作用するかどうかを予測することができます。  
適合率 (precision, 陽性的中率) とPPI Scoreの目安は以下の図に示されるような値となります。  
*   $E > 8$ を「PPI有り」と予測した場合のprecisionは約10%
*   $E > 10$ を「PPI有り」と予測した場合のprecisionは約50%
*   $E > 12$ を「PPI有り」と予測した場合のprecisionは約80%


![](http://drive.google.com/uc?export=view&id=1aVl9yRh-E4HXtn6AQ1fjrMb0M7Nx-wxH)
(from Ohue M, _et al_. _Jikkenigaku_, 37(9):1469, 2019.)